# Aroma Notebook for Colab

## Config

In [ ]:
#@title Configurations



#@markdown WebUI Basic Auth Info
#@markdown Note that the length of password **MUST BE** at least 8.
webui_username = 'asd' #@param {type:"string"}
webui_password = 'asdasdas' #@param {type:"string"}

#@markdown Daemon password
daemon_password = 'as' #@param {type:"string"}

#@markdown **NOTE** To get ngrok token, visit https://dashboard.ngrok.com/get-started/your-authtoken for ngrok token
ngrok_token = '' #@param {type:"string"}

#@markdown Aroma Repository
repo_url = 'https://github.com/lumiknit/aroma' #@param {type:"string"}

## Setup

In [ ]:
!curl -sL https://deb.nodesource.com/setup_18.x | sudo -E bash -
!apt-get install -y nodejs
!pip install pyngrok pytorch-lightning numpy pillow scipy ftfy transformers diffusers accelerate safetensors omegaconf huggingface_hub tensorrt

In [ ]:
# Clone repo and setup
%cd /
!rm -rf /ws
!git clone $repo_url /ws/aroma
%cd /ws/aroma
!git lfs install
!mkdir -p /ws/aroma/models /ws/aroma/archives /ws/aroma/outputs /ws/aroma/state

In [ ]:
# Change host and port to 0.0.0.0:9125
import json

config = {
    'webui': {
        'host': '0.0.0.0',
        'port': 32191,
        'username': webui_username,
        'password': webui_password,
        "model_download_presets": {
            "runwayml/stable-diffusion-v1-5": [""],
        }
    },
    'save_raw': False,
    'password': daemon_password,
    'init_values': {
        'params': {
            "prompt": "cat",
            "negative_prompt": "EasyNegative, veryBadImageNegative_v1.3, bad quality, low quality, bad anatomy, nsfw",
            'width': 512,
            'height': 512,
            "highres_fix": [
                {
                  "scale": 1.5,
                  "strength": 0.5
                }
            ]
        }
    }
}
config_json = json.dumps(config)

!echo '{config_json}' > /ws/aroma/config.json
!touch /ws/aroma/state/values.json

In [ ]:
# Install node packages
!cd /ws/aroma/ui && npm install -g npm && npm install
!node --version

# Run

In [ ]:
import os
import threading
from pyngrok import ngrok
ngrok.set_auth_token(ngrok_token)
public_url = ngrok.connect(32191, auth=f"{webui_username}:{webui_password}").public_url
print(f"{public_url}")
!node ui/main.js